<a href="https://colab.research.google.com/github/Ircken/Codigo/blob/main/juliaVDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalacion

In [1]:
# Installation cell
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

After you run the first cell (the the cell directly above this text), go to Colab's menu bar and select **Edit** and select **Notebook settings** from the drop down. Select *Julia 1.7* in Runtime type. You can also select your prefered harwdware acceleration (defaults to GPU). 

<br/>You should see something like this:

> ![Colab Img](https://raw.githubusercontent.com/Dsantra92/Julia-on-Colab/master/misc/julia_menu.png)

<br/>Click on SAVE
<br/>**We are ready to get going**





In [1]:
VERSION

v"1.7.2"

# Codigo

In [ ]:
import Pkg; Pkg.add("DataFrames")
import Pkg; Pkg.add("Statistics")
import Pkg; Pkg.add("CSV")
import Pkg; Pkg.add("Suppressor")
import Pkg; Pkg.add("Dates")
""

In [ ]:
using DataFrames
using Statistics
using CSV
using Suppressor
using Dates

In [ ]:
function inicializar()
    out = DataFrame(Hora = String[], kWhcount = Float64[], count = Int64[])
    rename!(out, [:"Hora", :"kWh*count", :"count"])

    for i in 0:23
        for j in 0:3
            append!(out[!, "Hora"],[hora(i, j)])
            append!(out[!, "kWh*count"],[0.0])
            append!(out[!, "count"],[0])
        end
    end    
    
    return out
end

inicializar (generic function with 1 method)

In [ ]:
function hora(i, j)      
    if i != 0
        if i < 10
            h = "0" * string(i)
        else
            h = string(i)
        end
    else
        h = "00"  
    end   

    if j != 0
        m = string(j*15)
    else
        m = "00"    
    end   
    
    return h * ":" * m * ":" * "00"
end

hora (generic function with 1 method)

In [ ]:
function sumar_columnas(out, df)
    
    if !(isempty(out)) || !(isempty(df))
            out[!, 2] .= out[!, 2] + df[!, 2]
            out[!, 3] .= out[!, 3] + df[!, 3]
        return out
    end
    
end

sumar_columnas (generic function with 1 method)

In [ ]:
function df_por_tarifa(df)

    aux = DataFrame()
    out = DataFrame(Hora = String[], kWhcount = Float64[], count = Int64[])
    rename!(out, [:"Hora", :"kWh*count", :"count"])

    for i in 0:23
        for j in 0:3
            horario = hora(i, j)
            aux = filter(:"Hora" => n -> n == horario, df)

            kWh = mean(aux[!, 2])
            count = length(aux[!, 2])
            prod = kWh*count

            #push!(aux, [horario prod count])
            append!(out[!, "Hora"],[horario])
            append!(out[!, "kWh*count"],[prod])
            append!(out[!, "count"],[count])        
        end
    end
    return out
    
end

df_por_tarifa (generic function with 1 method)

In [ ]:
# obtener tarifa
function t_n_f_index(archivo)

    open(archivo, "r") do f
        lines = readlines(f)

        tariff_index = findfirst(t -> occursin("Servicio:", t), lines) # indices de tarifa y fecha
                             
        # obtener tipo de tarifa
        tariff_vector = split(lines[tariff_index], ";")
        filter!(x -> x != " ", tariff_vector) # eliminar espacio vacio 
        filter!(x -> x != "", tariff_vector) # eliminar espacio vacio 
        if occursin("Medicion:", tariff_vector[2])
            tariff_vector[2] = "Medicion:"
        end
        
        # tarifa e indice de fecha
        return tariff_vector[2]
    end
    
end

t_n_f_index (generic function with 1 method)

In [ ]:
function limpiarTexto(f)

    lines = readlines(f)
    fecha_in = findfirst(t -> occursin("Fecha", t), lines)
    lines[end] = "" # eliminar ultima linea
    # si existe esa esctructura eliminarla
    if lines[end - 1] == ";;;;;;;;;;;;;;;;"
        lines[end - 1] = "" 
    end

    lines = lines[fecha_in: end] # obtener solo las lineas a partir de la fecha
    lines[1] = replace(lines[1], "\xe1" => "") # sustituir caracteres especiales  
    string = join(lines, "\n") # vector a string para leerlo como csv
    string = replace(string, ";;" => ";") # eliminar duplicados
    return string
    
end

limpiarTexto (generic function with 1 method)

In [ ]:
function primeraLimpiezaDF(df)
    
    # renombrar "Fecha hora" a "Fecha Hora"
    if "Fecha hora" in names(df)
        rename!(df, "Fecha hora" => "Fecha Hora")
    end
    select!(df, "Fecha Hora", "kWh") # eliminar las columnas no utiles
    dropmissing!(df, "Fecha Hora") # eliminar los missing
    dropmissing!(df, "kWh") # eliminar los missing
    
    return df
end

primeraLimpiezaDF (generic function with 1 method)

In [ ]:
function segundaLimpiezaDF(df)
    
    df[!,2] .= replace.(df[!,2], "," => ".") # coma a punto
    df[!,2] .= parse.(Float64, df[!,2]) # string a float

    transform!(df, :"Fecha Hora" => ByRow(x -> split(x, ' ')) => [:"Fecha", :"Hora"]) # separar fecha y hora
    df = df[:, [2, 4]] # dejar las columnas Fecha y Fecha Hora
    df[!, 1], df[!, 2] = df[!, 2], df[!, 1] # cambiar valores de las columnas
    rename!(df, [:"Hora", :"kWh"]) # renombrar columnas
    
    return df
end

segundaLimpiezaDF (generic function with 1 method)

In [ ]:
carpeta = "procesar"
if !isdir(carpeta)
    mkdir(carpeta)
end

"procesar"

In [ ]:
# obtener ubicacion de todos los archivos
archivos = Vector{String}()
# recorrer cada carpeta dentro de "carpeta"
for folder in readdir(carpeta)
    
    # recorrer cada archivo dentro de la carpeta dentro de "folder"
    for file in readdir(carpeta * "/" *folder)
        push!(archivos, "$carpeta/$folder/$file")
    end
    
end

In [ ]:
println(length(archivos))

0


In [ ]:
# obtener vectores de tarifa e indice de fecha
tariff = Vector{String}()

for i in 1:length(archivos)

    archivo = archivos[i] # ORDEN DE LECTURA
    aux = t_n_f_index(archivo)
    # guardar tarifa e indice de fecha
    push!(tariff, aux)

end

In [ ]:
tariff_set_list = collect(Set(tariff)) # lista de valores unicos
for i in tariff_set_list
    println("$i -> ", count(==(i), tariff))
end

In [ ]:
arch = findall(x -> occursin("Medicion:", x) , tariff)
cant_archivos = length(arch)
println(cant_archivos)
#println(arch[1:cant_archivos]) 
""

0


""

In [ ]:
# definir carpeta de salida
carpeta_salida = "excel_out"
if !isdir(carpeta_salida)
    mkdir(carpeta_salida)
end

"excel_out"

In [ ]:
# definir tiempo de salida
t = Dates.now() 
formatoTiempo = "$(Dates.hour(t))h$(Dates.second(t))m$(Dates.minute(t))s_$(Dates.Date(t))"
""

""

In [ ]:
# para cada tarifa
# recorrer elementos unicos de tarifa
@suppress begin
    for i in 1:length(tariff_set_list)
        
        global contador = 0
        # se opera sobre cada coincidencia de tarifa
        # sin procesar "Medicion:"
        if occursin("Medicion:", tariff_set_list[i]) != true
            global out = inicializar() # inicializar un df generico
            posicion_tarifas = findall(x -> x == tariff_set_list[i] , tariff) # posiciones de tarifa    

            # recorrer cada archivo de cada tarifa
            for indice_tarifa in posicion_tarifas
                archivo = archivos[indice_tarifa] # archivo con ese indice de tarifa
                
                    # abrir archivo
                    open(archivo, "r") do f

                        string = limpiarTexto(f) # preparar texto para df
                        df = CSV.read(IOBuffer(string), DataFrame; delim = ";", header = true) # string a csv                    
                        df = primeraLimpiezaDF(df)

                        if length(df[!, 1]) > 0
                            
                            global contador += 1
                            df = segundaLimpiezaDF(df)
                            df_out = df_por_tarifa(df) # df de solo 24h*(60min/1h)/15min = 96

                            # limpiar NaN
                            for col in eachcol(df_out)
                                replace!(col, NaN => 0.0)
                            end    

                            out = sumar_columnas(df_out, out) # sumar df especifico
                        
                        end 
                    end
            end
            nombre = "$(tariff_set_list[i])-$contador-$formatoTiempo" 
            CSV.write("$carpeta_salida/$nombre.csv", out) # guardar df por tarifa
        end
    end
end
""

""

---------------------------------------------------------------------------------------------------------------------------------------------------